# 웹 스크래핑(web scraping with python)
    어떠한 웹페이지의 HTML 내용을 분석하여 원하는 값을 추출해내는 것을 스크래핑(scraping)이라고 합니다.
    python의 기본패키지 및 BeautifulSoup을 이용해 daum.net의 검색결과 페이지를 스크래핑하는 예제 코드를 첨부합니다.

    이 코드를 실행하려면 먼저 BeautifulSoup 패키지를 설치해야 합니다.
    BeautifulSoup 패키지의 이름은 간단하게 'bs4' 입니다.

    그래서 윈도우 커맨드 창에서 'easy_install bs4' 라고 실행하면 패키지를 설치할 수 있습니다.
    그런데 easy_install 를 설치하지 않았다면 설치해야 합니다.
    
    
    bs4 설치방법
    1) easy_install을 설치한다 방법은 http://blog.colab.kr/11 참고.
    2) easy_install 이 설치된 후에 윈도우 command 창에서 다음과 같이 실행한다.
    $ easy_install bs4

In [1]:
# coding: utf-8
import urllib2
import urllib
import urlparse
import time
from bs4 import BeautifulSoup as bs


USER_AGENT_CHROME = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.37 Safari/537.36"
USER_AGENT_FIREFOX = "Mozilla/5.0 (X11; U; FreeBSD i386; en-US; rv:1.7.8) Gecko/20050609 Firefox/1.0.4"
AGENTS = [USER_AGENT_FIREFOX, USER_AGENT_CHROME]
agent_cnt = 0

In [3]:
def fetch(web_url):
  '''
  주어진 웹페이지의 HTML 내용을 읽어옴
  :param web_url: the url of a webpage
  :return: HTML contents (문자열)
  '''
  global agent_cnt
  agent_cnt += 1
  try:
    req = urllib.Request(web_url)
    req.add_header("User-Agent", AGENTS[agent_cnt % 2])
    req.add_header("Connection", "Keep-Alive")
    #print req.headers
    f = urllib.urlopen(req)
    html = f.read()
    #print len(html)
    return html
  except urllib.HTTPError, e:
    print e.code
    print e.msg
    return ''


In [5]:
def get_links(html, org_url):
  '''
  HTML 문서에 포함된 link 주소(URL)들의 목록을 리턴함
  :param html: HTML 문서 내용
  :param org_url: HTML 문서의 원래 웹주소. 문서내의 링크가 상대경로인 경우에 원래 웹주소와의 결합이 필요함(urljoin)
  :return: the list of links
  '''
  soup = bs(html, 'lxml')
  links = [urlparse.urljoin(org_url, link.get('href')) for link in soup.find_all('a')]
  links = [link for link in links if link.startswith('http')]
  return links


In [6]:
def get_text(html):
  '''
  HTML 문서에서 script, style 부분을 제외한 텍스트 내용을 리턴함.
  :param html:
  :return:
  '''
  soup = bs(html, 'lxml')
  for s in soup('script'): s.extract()  # script, 자바스크립트 등 제거
  for s in soup('style'): s.extract()  # CSS 등 style 코드 제거
  for br in soup.find_all("br"):
    br.replace_with("\n")

  cleanedlinelist = [' '.join(s.split()) for s in soup.strings]
  filtered =  filter(len, cleanedlinelist)
  return '\n'.join(filtered)  # 화면에 출력해보려면  join()을 사용하여 리스트를  문자열로 변환.
  #return soup.get_text()

In [7]:
def convert_to_clean_lines(text):
  '''
  글 내용 중에 불필요한 공백을 제거함.
  1) 연속된 여러 공백(space,tab)은 하나의 공백(space)으로 변환,
  2) 빈 라인(공백만 있는 라인)은 제거
  :param text: 텍스트
  :return: 깔끔한 텍스트
  '''


  res = [' '.join(line.split()) for line in text.splitlines()] # 각 라인의 앞뒤 공백 제거, 중간 공백둘은 한개의 space 로 변환. 리스트
  res = filter(len, res) # 빈 라인(길이가 0인 것) 제거
  return '\n'.join(res)  # 뉴라인 포함된 문자열로 다신 만듦

In [8]:
def main(weburl):
  html = fetch(weburl)
  text = get_text(html)
  print text
  #cleaned =  convert_to_clean_lines(text)
  #print cleaned

In [9]:
def build_daum_search_url(searchword):
  '''
  http://search.daum.net/search?
  w=tot&
  DA=YZR&
  t__nil_searchbox=btn&
  sug=&
  sugo=&
  q=%EC%8A%AC%EB%A1%9C%EC%9A%B0%EC%BA%A0%ED%8D%BC%EC%8A%A4&
  tltm=kk43

  :param searchword:
  :return:
  '''
  kv = {
    'w': 'tot',
    'q': searchword
  }
  param = urllib.urlencode(kv)
  return 'http://search.daum.net/search?' + param

In [10]:
def search_daum(word):
  searchurl = build_daum_search_url(word)
  html = fetch(searchurl)
  links = get_links(html, searchurl)
  print '\n'.join(links)
  text = get_text(html)
  print text

In [12]:
#if __name__ == '__main__':
#  #main(myaddr)
search_daum('슬로우캠퍼스')

AttributeError: 'module' object has no attribute 'HTTPError'